In [1]:
import pandas as pd
import numpy as np
import math

from collections import Counter

import scipy as sp
#from scipy.stats import nanmean
#from scipy.stats import nanstd

import warnings
warnings.filterwarnings('ignore')


## Import data

In [5]:
round_data = pd.read_csv('./all_studies_round_data.csv')
round_data.columns
round_data['current_rev_score'] = round_data['cumulative_score'] - round_data['increment']
round_data['current_ind_score'] = round_data['cumulative_ind_score'] - round_data['increment']

round_data['half_cumulative'] = round_data['cumulative_score']

round_data.columns

Index(['game_id', 'condition', 'round_index', 'correct_answer',
       'plot_sequence', 'player_id', 'cumulative_score', 'difficulty',
       'revised_guess', 'revised_error', 'alter_1', 'alter_2', 'alter_3',
       'increment', 'increment_color', 'in_degree', 'study',
       'round_after_shock', 'quarter', 'task_instance_avg_error',
       'revised_error_relative2solo', 'independent_error_relative2solo',
       'independent_guess', 'independent_error', 'half', 'best_player',
       'ind_score', 'cumulative_ind_score', 'final_score', 'final_ind_score',
       'improvement', 'group', 'best_player_initial', 'best_player_revised',
       'current_rev_score', 'current_ind_score', 'half_cumulative'],
      dtype='object')

In [10]:
#count player id unique id
round_data['player_id'].nunique() 

1421

In [15]:
# check independent error mean for condition solo_feedback at round 1
round_data[(round_data['round_index'] == 1) & (round_data['condition'] == 'solo_feedback')]['independent_error'].mean()

np.float64(0.28602510460251046)

In [4]:
round_data["condition"].value_counts()


condition
solo_feedback            4820
dynamic                  4800
static                   4800
dynamic_no_feedback      3600
dynamic_full_feedback    3600
dynamic_self_feedback    3600
solo_no_feedback         3240
Name: count, dtype: int64

In [55]:
round_data = round_data[round_data["condition"] == "dynamic"]
round_data.head()



,game_id,condition,round_index,correct_answer,plot_sequence,player_id,cumulative_score,difficulty,revised_guess,revised_error,...,cumulative_ind_score,final_score,final_ind_score,improvement,group,best_player_initial,best_player_revised,current_rev_score,current_ind_score,half_cumulative
0,103,dynamic,1,0.09,easy->hard,y7gd5boo3cl0s6ndywyv6kkkgfx5tf0msk4jcmaqwwlwcm...,0.96,easy,0.05,0.04,...,0.96,16.21,15.44,0.00,1,False,False,0.0,0.00,0.96
4,82,dynamic,1,0.09,hard->medium,qsx1yagowvkapla9t5e5w3e32xluu11zomczvwon816cd4...,0.71,hard,0.38,0.29,...,0.69,17.28,17.06,-0.02,1,False,False,0.0,-0.02,0.71
12,92,dynamic,1,0.09,easy->hard,z8ht5amxkrscf26ws933bczcmzydhbu81dj0a5xhcwm6nb...,0.49,easy,0.60,0.51,...,0.58,16.24,12.26,0.09,1,False,False,0.0,0.09,0.49
29,92,dynamic,1,0.09,hard->medium,3gcu8djert6f3zr6nrier80jn3ck7f9bhjpz0rcgzciesa...,0.84,hard,0.25,0.16,...,0.63,18.00,17.09,-0.21,1,False,False,0.0,-0.21,0.84
31,92,dynamic,1,0.09,hard->medium,xs8oemk2r3v4mce9fn8kn9umu2ycsr0am4ob9101ej4386...,0.69,hard,0.40,0.31,...,0.73,17.41,15.29,0.04,1,False,False,0.0,0.04,0.69


In [56]:
game = round_data[round_data["game_id"] == "103"]
# sort by round
game = game.sort_values(by = "round_index")

# only shows only rows for a random player_id
game["player_id"].unique()

array(['y7gd5boo3cl0s6ndywyv6kkkgfx5tf0msk4jcmaqwwlwcmq3oq7cghbzcrh6fd1yc44uthe14rqbvn7y',
       'umbd5bbutvp8l2ylfsb70rinncc2ob2dw4jt1ld7xg14y9yh3zjj0tdsr8xwoah3xonsbyc87i5g9g11',
       'ssmy6zq5kmkhdf2tccdqk7xc3lsunz9sbyhmv1ogz26dimu3pbm4aa1x08g4ktclsxmok5inj50phqbu',
       'ydi21ho6mgiz04gre8u5furhmefzawjybw2lfetbwd0lbozls16aux94xrm9hf0v8ozzsjllu55e0phl',
       'vqbqm4hak8eoatuocf50us24sj5s02y8jp9ii1nbbmd4ekh00ojni2sz6bdxcxjg819lro7qvujtsk9u',
       '8n2cgndwy1sf2zq68pwsjvxg2bmev1donzdtu10k6vp912jzva6e8arw2ycx6heq8uhy352y40ysfo4m',
       'dkdrz2tbqch1humah47et3f5p5tb3u05he6b5xyo4mb08a9mg3rsr5i76z3d76brm19fekl7r6pxv54i',
       'u7keoqnqrhw3kx18cnkh738ocuzpiup8t0lkmtv7pqwmwtxisqdj8byts2uqmlfrlh33b6ia8fzmadsz',
       'ha9zyrvqhfdedi6pdxyj1kx3ztn6eze6ls4ry1r5vgf4cgp6nzdsa0joqs85xwwytstvyhi58hssu8ld',
       '26plqyod4ouk75csq5nm82yal11e1phdf8yh5kv7gbkrzbimrc6rvjuxlu1jl395j4qm7o3mupxgjfp4',
       'dc0u4lzmq383j31e5lkhv76ocvpb2m8zvm8x46nqohhfhfgzvpudr5du1huxooj56hghjqrnx58thchx',

In [57]:
game = game[ game["player_id"] == "y7gd5boo3cl0s6ndywyv6kkkgfx5tf0msk4jcmaqwwlwcmq3oq7cghbzcrh6fd1yc44uthe14rqbvn7y"].reset_index()

#remove  columns player_id alter_1, alter_2, alter_3
game = game.drop(columns = ["alter_1", "alter_2", "alter_3"])
# remove duplicates
game = game.drop_duplicates()
# save to csv 
game.to_csv('./player_dynamic_game.csv', index=False)

In [58]:
game.to_csv('./new_player_dynamic_game.csv', index=False)

In [50]:
round_data = round_data[["condition", "player_id", "round_index", "correct_answer", "independent_guess", "revised_guess", "cumulative_score", "difficulty", "increment", ]]

#sort by player_id and round_index
round_data = round_data.sort_values(by=["player_id", "round_index"])



round_data.head()

,condition,player_id,round_index,correct_answer,independent_guess,revised_guess,cumulative_score,difficulty,increment
531,dynamic,01evj2yyd6jvqvnzroopn7je1bib3qqilxzudxn9aort7u...,1,0.09,0.05,0.05,0.96,easy,0.96
1675,dynamic,01evj2yyd6jvqvnzroopn7je1bib3qqilxzudxn9aort7u...,2,0.78,0.86,0.77,1.95,easy,0.99
3633,dynamic,01evj2yyd6jvqvnzroopn7je1bib3qqilxzudxn9aort7u...,3,0.91,0.80,0.87,2.91,easy,0.96
5451,dynamic,01evj2yyd6jvqvnzroopn7je1bib3qqilxzudxn9aort7u...,4,0.47,0.28,0.22,3.66,easy,0.75
5945,dynamic,01evj2yyd6jvqvnzroopn7je1bib3qqilxzudxn9aort7u...,5,0.08,0.04,0.04,4.62,easy,0.96


In [ ]:
round_data['half_cumulative_2nd'] = round_data[round_data.round_index>10].groupby('player_id')['increment'].transform(pd.Series.cumsum)

round_data.half_cumulative_2nd.fillna(round_data.half_cumulative, inplace=True)
del round_data['half_cumulative']
round_data = round_data.rename(columns={"half_cumulative_2nd": "half_cumulative"})

round_data[round_data.player_id == "rFbSt2rKkEKdinL88"][['player_id',
            'round_index','increment','cumulative_score','final_score', 'half_cumulative']].sort_values("round_index", ascending=True)


,player_id,round_index,increment,cumulative_score,final_score,half_cumulative
954,rFbSt2rKkEKdinL88,1,0.70,0.70,15.02,0.70
1917,rFbSt2rKkEKdinL88,2,0.48,1.18,15.02,1.18
2924,rFbSt2rKkEKdinL88,3,0.92,2.10,15.02,2.10
5559,rFbSt2rKkEKdinL88,4,0.84,2.94,15.02,2.94
5885,rFbSt2rKkEKdinL88,5,0.64,3.58,15.02,3.58
7708,rFbSt2rKkEKdinL88,6,0.90,4.48,15.02,4.48
9393,rFbSt2rKkEKdinL88,7,0.83,5.31,15.02,5.31
10559,rFbSt2rKkEKdinL88,8,0.96,6.27,15.02,6.27
11714,rFbSt2rKkEKdinL88,9,0.96,7.23,15.02,7.23
13513,rFbSt2rKkEKdinL88,10,0.68,7.91,15.02,7.91


### Compute additional columns:

In [4]:
def get_score_and_cum_score(round_data):
    round_data = round_data.sort_values(by='round_index')
    round_data['ind_score'] = 1-abs(round_data.independent_guess - round_data.correct_answer)
    round_data['cumulative_ind_score'] = round_data.groupby('player_id')['ind_score'].transform(pd.Series.cumsum)
    round_data['final_score'] = round_data.groupby('player_id')['cumulative_score'].transform(pd.Series.max)
    round_data['final_ind_score'] = round_data.groupby('player_id')['cumulative_ind_score'].transform(pd.Series.max)

    return  round_data

round_data = get_score_and_cum_score(round_data)
round_data.columns

Index(['game_id', 'condition', 'round_index', 'correct_answer',
       'plot_sequence', 'player_id', 'cumulative_score', 'difficulty',
       'revised_guess', 'revised_error', 'alter_1', 'alter_2', 'alter_3',
       'increment', 'increment_color', 'in_degree', 'study',
       'alter_1_revised_guess', 'alter_2_revised_guess',
       'alter_3_revised_guess', 'alter_1_independent_guess',
       'alter_2_independent_guess', 'alter_3_independent_guess',
       'round_after_shock', 'quarter', 'task_instance_avg_error',
       'revised_error_relative2solo', 'independent_error_relative2solo',
       'independent_guess', 'independent_error', 'half', 'best_player',
       'ind_score', 'cumulative_ind_score', 'final_score', 'final_ind_score',
       'improvement', 'group', 'best_player_initial', 'best_player_revised',
       'current_rev_score', 'current_ind_score', 'half_cumulative'],
      dtype='object')

# 2. Make game data

In [5]:
def game_data_with_top_cols(round_data,n_boot):
    
    game_data_list = []
    
    ####### make all conditions exept solos
    for condition in ['dynamic_no_feedback','dynamic_self_feedback','dynamic_full_feedback',  'static', 'dynamic']:
        print('aggregating condition ', condition)
        for game in round_data[round_data.condition == condition].game_id.unique():
            for rid in range(1,20+1):
                this_game_round_data = round_data[(round_data.condition==condition) &
                                                 (round_data.round_index==rid) &
                                                 (round_data.game_id == game)]
                
                group_guess_rev = this_game_round_data.revised_guess.mean(skipna=True)
                group_guess_ind = this_game_round_data.independent_guess.mean(skipna=True)
                correct_an =  this_game_round_data.correct_answer.iloc[0]

                game_data = {'game_id':game, 'condition':condition,
                             'round_index':rid,
                             'independent_wisdom':group_guess_ind,
                             'revised_wisdom':group_guess_rev,
                             'revised_wisdom_error': abs( group_guess_rev - correct_an ),
                             'independent_wisdom_error': abs(group_guess_ind - correct_an ),
                             'correct_answer': correct_an,
                             'task_instance_avg_error': this_game_round_data.task_instance_avg_error.iloc[0]
                            }
                
                ## top_k errors for the trade off plot
                for k in range(1,12+1): 
                    # use [this_game_round_data.round_index ==rid] for issue with duplicated ids because of sampling
                    top_k_an = this_game_round_data[this_game_round_data.round_index ==rid].sort_values("final_ind_score", ascending=False).head(k).revised_guess.mean()
                    game_data['top_'+str(k)+'_ind'] = abs(top_k_an - correct_an)
                    
                    top_k_an = this_game_round_data[this_game_round_data.round_index ==rid].sort_values("final_score", ascending=False).head(k).revised_guess.mean()
                    game_data['top_'+str(k)+'_rev'] = abs(top_k_an - correct_an)
                    
                    
                    top_k_an = this_game_round_data[this_game_round_data.round_index ==rid].sort_values("current_ind_score", ascending=False).head(k).revised_guess.mean()
                    game_data['top_'+str(k)+'_ind_live'] = abs(top_k_an - correct_an)

                    top_k_an = this_game_round_data[this_game_round_data.round_index ==rid].sort_values("current_rev_score", ascending=False).head(k).revised_guess.mean()
                    game_data['top_'+str(k)+'_rev_live'] = abs(top_k_an - correct_an)
                    
                    if rid < 11:
                        this_game_quarter_data = round_data[(round_data.condition==condition) &
                                 (round_data.round_index==5) &
                                 (round_data.game_id == game)] ##this gives me the first quarter data
                    else:
                        this_game_quarter_data = round_data[(round_data.condition==condition) &
                                                            (round_data.round_index==15) &
                                                            (round_data.game_id == game)] ##this gives me the second quarter data

                    top_k_players = this_game_quarter_data.sort_values("half_cumulative", 
                                                                       ascending=False).head(k).player_id.tolist()#these are the top k players in the quarter

                    #print(top_k_players)
                    top_k_an = this_game_round_data[(this_game_round_data.round_index ==rid) & 
                                                    (this_game_round_data.player_id.isin(top_k_players))].revised_guess.mean()
                    game_data['top_'+str(k)+'_rev_ex_ante'] = abs(top_k_an - correct_an)

                    
#                     top_k_players = this_game_quarter_data.sort_values("current_ind_score", 
#                                                                        ascending=False).head(k).player_id.tolist()#these are the top k players in the quarter

#                     #print(top_k_players)
#                     top_k_an = this_game_round_data[(this_game_round_data.round_index ==rid) & 
#                                                     (this_game_round_data.player_id.isin(top_k_players))].revised_guess.mean()
#                     game_data['top_'+str(k)+'_ind_ex_ante'] = abs(top_k_an - correct_an)


                    
                game_data_list.append(game_data)
                
    
    ######### make bootstrap solo rows
    
    #select 12
    bootstrap_conditions  = ['solo_feedback', 'solo_no_feedback']
    for solo_condition in bootstrap_conditions:
        seq = {0: round_data[(round_data.condition==solo_condition) & (round_data.plot_sequence == 'easy->hard')]['player_id'].unique(),
               1: round_data[(round_data.condition==solo_condition) & (round_data.plot_sequence == 'medium->easy')]['player_id'].unique(),
               2: round_data[(round_data.condition==solo_condition) & (round_data.plot_sequence == 'hard->medium')]['player_id'].unique()
              }
        print("Solo condition bootstrapping now is", solo_condition)
        for j in range(0,n_boot):
            print("Boot      ", j, end='\r')
            this_game_round_data = None
            #print ('bootstrap ', j)
            this_game_ids = list(np.random.choice(seq[0],size= 4, replace= True)) + list(np.random.choice(seq[1],size= 4, replace= True)) + list(np.random.choice(seq[2],size= 4, replace= True))
            this_game_round_data = round_data[round_data.player_id.isin(this_game_ids)]
            this_game_round_data['game_id '] = j


            #make sure that users are sampled once otherwise append them
            repeated_sample = Counter(this_game_ids)
            #return repeated_sample

            repeated_sample_counts = list(repeated_sample.values())

            if sum(i > 1 for i in repeated_sample_counts) > 0:    
                #print('number of repeated player', sum(i > 1 for i in repeated_sample_counts) )
                for repeated_user in repeated_sample.keys():
                    repetitions = (repeated_sample[repeated_user]-1)
                    if repetitions > 0:
                        this_game_round_data = this_game_round_data.append([this_game_round_data[this_game_round_data.player_id==repeated_user]]*repetitions)


            #print(len(this_game_round_data))
            #this_game_round_data = add_rank_solo(this_game_round_data)
            #print(this_game_round_data.round_index)
            for rid in range(1,20+1):
                guess_rev = this_game_round_data[this_game_round_data.round_index ==rid].revised_guess.mean(skipna=True)
                guess_ind = this_game_round_data[this_game_round_data.round_index ==rid].independent_guess.mean(skipna=True)
                #print(this_game_round_data)
                #print(this_game_round_data[this_game_round_data.round_id ==rid].correct_answer)
                correct_an =  this_game_round_data[this_game_round_data.round_index ==rid].correct_answer.iloc[0]
                game_data = {'game_id':str(j)+'_'+solo_condition,
                             'condition':solo_condition,
                             'round_index':rid,
                             'independent_wisdom':guess_ind,
                             'revised_wisdom':guess_rev,
                             'revised_wisdom_error': abs( guess_rev - correct_an ),
                             'independent_wisdom_error': abs(guess_ind - correct_an ),
                             'correct_answer': correct_an,
                             'task_instance_avg_error': this_game_round_data.task_instance_avg_error.iloc[0]
                            }
                ## top_k errors for the trade off plot
                for k in range(1,12+1):   
                    # use [this_game_round_data.round_index ==rid] for issue with duplicated ids because of sampling
                    #access to underlaying top-k (ex-post)
                    top_k_an = this_game_round_data[this_game_round_data.round_index ==rid].sort_values("final_ind_score", ascending=False).head(k).revised_guess.mean()
                    game_data['top_'+str(k)+'_ind'] = abs(top_k_an - correct_an)
                    
                    top_k_an = this_game_round_data[this_game_round_data.round_index ==rid].sort_values("final_score", ascending=False).head(k).revised_guess.mean()
                    game_data['top_'+str(k)+'_rev'] = abs(top_k_an - correct_an)
                    
                    
                    ##live means (score up to now); 
                    top_k_an = this_game_round_data[this_game_round_data.round_index ==rid].sort_values("current_ind_score", ascending=False).head(k).revised_guess.mean()
                    game_data['top_'+str(k)+'_ind_live'] = abs(top_k_an - correct_an)

                    top_k_an = this_game_round_data[this_game_round_data.round_index ==rid].sort_values("current_rev_score", ascending=False).head(k).revised_guess.mean()
                    game_data['top_'+str(k)+'_rev_live'] = abs(top_k_an - correct_an)
                    

                    ##this is for the quarter ex ante approach:
                    #get performance estimates in first quarter, and then look at perofmrna in second quarter.
                    #Then do the same in third + fourth quarters
                    if rid < 11:
                        this_game_quarter_data = this_game_round_data[(this_game_round_data.round_index==5)] ##this gives me the first quarter data
                    else:
                        this_game_quarter_data = this_game_round_data[(this_game_round_data.round_index==15)] ##this gives me the first quarter data

                    top_k_players = this_game_quarter_data.sort_values("half_cumulative", 
                                                                       ascending=False).head(k).player_id.tolist()#these are the top k players in the quarter

                    #print(top_k_players)
                    top_k_an = this_game_round_data[(this_game_round_data.round_index ==rid) & 
                                                    (this_game_round_data.player_id.isin(top_k_players))].revised_guess.mean()
                    game_data['top_'+str(k)+'_rev_ex_ante'] = abs(top_k_an - correct_an)
                    
                    
                    
                    ##based on independent score
#                     top_k_players = this_game_quarter_data.sort_values("current_ind_score", 
#                                                                        ascending=False).head(k).player_id.tolist()#these are the top k players in the quarter

#                     #print(top_k_players)
#                     top_k_an = this_game_round_data[(this_game_round_data.round_index ==rid) & 
#                                                     (this_game_round_data.player_id.isin(top_k_players))].revised_guess.mean()
#                     game_data['top_'+str(k)+'_ind_ex_ante'] = abs(top_k_an - correct_an)

                    


                game_data_list.append(game_data)
                
    
    
    game_data = pd.DataFrame(game_data_list)     #*

    
    #set time after shock .. 
    game_data['round_after_shock'] = game_data.round_index
    game_data['round_after_shock'] = np.where(game_data['round_index'] > 10 ,game_data['round_index']-10,game_data['round_after_shock']) 
    
    
    #adding quarters
    game_data['quarter'] = None

    game_data.loc[game_data['round_index'] < 6,'quarter'] = 1
    game_data.loc[(game_data['round_index'] >= 6) & (game_data['round_index'] < 11),'quarter'] = 2
    game_data.loc[(game_data['round_index'] >= 11) & (game_data['round_index'] < 16),'quarter'] = 3
    game_data.loc[game_data['round_index'] >= 16,'quarter'] = 4


    game_data['half'] = None    
    game_data.loc[game_data.round_index < 11,'half'] = 1
    game_data.loc[game_data.round_index > 10,'half'] = 2

        
    return game_data

In [6]:
# compute df
game_rounds =  game_data_with_top_cols( round_data ,   300 )

# add cols
game_rounds['game_condition_id'] = game_rounds['game_id'].map(str) + game_rounds['condition'].map(str)

# prints
print("# of games per condition:\n", game_rounds[['game_condition_id','condition' ]].drop_duplicates()['condition'].value_counts())
game_rounds.head()

aggregating condition  dynamic_no_feedback
aggregating condition  dynamic_self_feedback
aggregating condition  dynamic_full_feedback
aggregating condition  static
aggregating condition  dynamic
Solo condition bootstrapping now is solo_feedback
Solo condition bootstrapping now is solo_no_feedback
# of games per condition:
 solo_feedback            300
solo_no_feedback         300
dynamic                   20
static                    20
dynamic_self_feedback     15
dynamic_full_feedback     15
dynamic_no_feedback       15
Name: condition, dtype: int64


,game_id,condition,round_index,independent_wisdom,revised_wisdom,revised_wisdom_error,independent_wisdom_error,correct_answer,task_instance_avg_error,top_1_ind,...,top_11_rev_ex_ante,top_12_ind,top_12_rev,top_12_ind_live,top_12_rev_live,top_12_rev_ex_ante,round_after_shock,quarter,half,game_condition_id
0,LtnXEGCJzhBnyARpj,dynamic_no_feedback,1,0.486667,0.570000,0.480000,0.396667,0.09,0.274051,0.68,...,0.460000,0.480000,0.480000,0.480000,0.480000,0.480000,1,1,1,LtnXEGCJzhBnyARpjdynamic_no_feedback
1,LtnXEGCJzhBnyARpj,dynamic_no_feedback,2,0.724167,0.807500,0.027500,0.055833,0.78,0.229560,0.33,...,0.020000,0.027500,0.027500,0.027500,0.027500,0.027500,2,1,1,LtnXEGCJzhBnyARpjdynamic_no_feedback
2,LtnXEGCJzhBnyARpj,dynamic_no_feedback,3,0.596667,0.487500,0.422500,0.313333,0.91,0.304500,0.26,...,0.398182,0.422500,0.422500,0.422500,0.422500,0.422500,3,1,1,LtnXEGCJzhBnyARpjdynamic_no_feedback
3,LtnXEGCJzhBnyARpj,dynamic_no_feedback,4,0.489167,0.470833,0.000833,0.019167,0.47,0.194340,0.08,...,0.010000,0.000833,0.000833,0.000833,0.000833,0.000833,4,1,1,LtnXEGCJzhBnyARpjdynamic_no_feedback
4,LtnXEGCJzhBnyARpj,dynamic_no_feedback,5,0.439167,0.408333,0.328333,0.359167,0.08,0.219937,0.22,...,0.283636,0.328333,0.328333,0.328333,0.328333,0.328333,5,1,1,LtnXEGCJzhBnyARpjdynamic_no_feedback


<br>
<br>
<br>
<br>

In [7]:
game_rounds.loc[game_rounds.condition.isin(['static','dynamic','solo_feedback']),'study'] = 1
game_rounds.loc[~game_rounds.condition.isin(['static','dynamic','solo_feedback']),'study'] = 2

In [8]:
# game_rounds['task_instance_avg_error'] = np.nan
# for study in [1,2]:
#     for rid in game_rounds.round_index.unique():
#         if study == 1:
#             bools = (game_rounds.condition=='solo_feedback') & (game_rounds.round_index == rid) & (game_rounds.study == study)
#         else:
#             bools = (game_rounds.condition=='solo_no_feedback') & (game_rounds.round_index == rid) & (game_rounds.study == study)
                        
                
#         game_rounds.loc[
#             (game_rounds['round_index'] == rid) &
#             (game_rounds.study == study),
#             'task_instance_avg_error'] = game_rounds[bools].revised_wisdom_error.mean(skipna=True)


# ##relative errors to solo
# game_rounds['revised_error_relative2solo'] = game_rounds.revised_wisdom_error - game_rounds.task_instance_avg_error
# game_rounds['independent_error_relative2solo'] = game_rounds.independent_wisdom_error - game_rounds.task_instance_avg_error


#game_rounds['plot_avg_error'] = None
#game_rounds['revised_error_relative2solo'] = game_rounds['revised_wisdom_error']

# for rid in range(1,20+1):
#     game_rounds[game_rounds.round_index==rid]['revised_error_relative2solo'] = 
#     game_rounds[game_rounds.round_index==rid]['revised_error_relative2solo'] - game_rounds[(game_rounds.condition=='solo') &(game_rounds.round_index==rid)]['revised_error_relative2solo'].mean()

study_1_round_error = {}
for rid in range(1,20+1):
    study_1_round_error[rid] = 0
    study_1_round_error[rid]  = game_rounds[ (game_rounds.study==1) & (game_rounds.condition == "solo_feedback") 
                                            & (game_rounds.round_index==rid)]['revised_wisdom_error'].mean()

study_2_round_error = {}
for rid in range(1,20+1):
    study_2_round_error[rid] = 0
    study_2_round_error[rid]  = game_rounds[(game_rounds.study==2)&(game_rounds.condition == "solo_no_feedback") 
                                            & (game_rounds.round_index==rid)]['revised_wisdom_error'].mean()

for i, row in game_rounds.iterrows():
    if row.study == 1:
        game_rounds.set_value(i,'revised_error_relative2solo',
                              (row['revised_wisdom_error']-study_1_round_error[row['round_index']]))

    else:
        game_rounds.set_value(i,'revised_error_relative2solo',
                              (row['revised_wisdom_error']-study_2_round_error[row['round_index']]))

game_rounds.loc[np.isinf(game_rounds.revised_error_relative2solo), 'revised_error_relative2solo'] = np.nan


In [9]:

game_rounds.to_csv('./all_studies_game_data_revision.csv')